In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('demand-forecasting-kernels-only/train.csv')

In [3]:
len(data)

913000

In [3]:
sales = data.loc[data.item == 5].loc[data.store == 1].sales

In [105]:
sales = sales/sales.max()

In [106]:
import plotly.express as px
import plotly.colors as c
fig = px.scatter(sales)
fig.show()

In [34]:
##################################################################################################################

In [100]:
# univariate lstm example
from numpy import array
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Bidirectional, Input, GRU, TimeDistributed, Conv1D, Flatten, MaxPooling1D
from keras.layers import Dense
import tensorflow as tf
import numpy as np

In [107]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X).astype('float32'), np.array(y).astype('float32')

X, y = split_sequence(list(sales), 500)
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [108]:
model = Sequential()
model.add(LSTM(265, input_shape=(500,1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=15)

Epoch 1/15
42/42 [==============================] - 3s 28ms/step - loss: 0.0630
Epoch 2/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0211
Epoch 3/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0186
Epoch 4/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0196
Epoch 5/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0183
Epoch 6/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0194
Epoch 7/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0191
Epoch 8/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0185
Epoch 9/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0193
Epoch 10/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0181
Epoch 11/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0188
Epoch 12/15
42/42 [==============================] - 1s 27ms/step - loss: 0.0173
Epoch 13/15
42/42 [==================

In [109]:
model = Sequential()
model.add(LSTM(265, return_sequences=True, input_shape=(500,1)))
model.add(LSTM(265))
model.add(Dense(1, activation = 'gelu'))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=15)

Epoch 1/15
42/42 [==============================] - 5s 62ms/step - loss: 0.0526
Epoch 2/15
42/42 [==============================] - 3s 62ms/step - loss: 0.0208
Epoch 3/15
42/42 [==============================] - 3s 61ms/step - loss: 0.0181
Epoch 4/15
42/42 [==============================] - 3s 62ms/step - loss: 0.0185
Epoch 5/15
42/42 [==============================] - 3s 61ms/step - loss: 0.0204
Epoch 6/15
42/42 [==============================] - 3s 61ms/step - loss: 0.0185
Epoch 7/15
42/42 [==============================] - 3s 61ms/step - loss: 0.0187
Epoch 8/15
42/42 [==============================] - 3s 61ms/step - loss: 0.0179
Epoch 9/15
42/42 [==============================] - 3s 61ms/step - loss: 0.0169
Epoch 10/15
42/42 [==============================] - 3s 61ms/step - loss: 0.0175
Epoch 11/15
42/42 [==============================] - 3s 61ms/step - loss: 0.0196
Epoch 12/15
42/42 [==============================] - 3s 61ms/step - loss: 0.0192
Epoch 13/15
42/42 [==================

In [110]:
model = Sequential()
model.add(Bidirectional(LSTM(265, input_shape=(500,1))))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=15)

Epoch 1/15
42/42 [==============================] - 5s 53ms/step - loss: 0.0582
Epoch 2/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0214
Epoch 3/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0190
Epoch 4/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0180
Epoch 5/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0202
Epoch 6/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0186
Epoch 7/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0195
Epoch 8/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0181
Epoch 9/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0182
Epoch 10/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0179
Epoch 11/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0181
Epoch 12/15
42/42 [==============================] - 2s 52ms/step - loss: 0.0197
Epoch 13/15
42/42 [==================

In [112]:
LAYER_SIZE = 124

input = Input(shape=(None,500,1),name = 'input')

conv_out = TimeDistributed(Conv1D(filters=64, kernel_size=7, activation='gelu'))(input)

flat = TimeDistributed(Flatten())(conv_out)

#gru_out = GRU(LAYER_SIZE)(input)
gru_out = GRU(LAYER_SIZE)(flat)                           

dense = Dense(1, activation = 'gelu')(gru_out)

In [113]:
model = Model(inputs=[input], outputs=[dense])
print(model.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None, 500, 1)]    0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, None, 494, 64)     512       
_________________________________________________________________
time_distributed_18 (TimeDis (None, None, 31616)       0         
_________________________________________________________________
gru_9 (GRU)                  (None, 124)               11808024  
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 125       
Total params: 11,808,661
Trainable params: 11,808,661
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X, y = split_sequence(list(sales), 500)

X = X.reshape((X.shape[0], X.shape[1], n_features))

In [114]:
model.compile(optimizer = 'adam', 
              loss='mse')

history = model.fit(x = X,
                    y = y,
                    epochs=15,
                    validation_split=0.05,
                    verbose=1)

Epoch 1/15


ValueError: in user code:

    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\aleu2

In [101]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)

ValueError: in user code:

    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\aleu2\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\aleu2